In [101]:
import numpy as np
import pandas as pd
from scipy import stats

In [140]:
city = 'cali'

In [141]:
def percentile( n ):
    def percentile_( x ):
        return np.percentile( x, n )
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [142]:
time = pd.read_csv( './data/meta/time.csv' )

In [143]:
time[ 'date' ] = pd.to_datetime( time[ 'date' ], format = '%d/%m/%Y' )

In [144]:
time.dtypes

date    datetime64[ns]
time             int64
dtype: object

In [145]:
time.head()

,date,time
0,2020-03-14,1
1,2020-03-15,2
2,2020-03-16,3
3,2020-03-17,4
4,2020-03-18,5


In [146]:
filename = ''
if city == 'bogota':
    filename = 'BOG'
elif city == 'barranquilla':
    filename = 'BQUILLA'
elif city == 'cali':
    filename = 'CALI'
    
meta = pd.read_csv( './data/meta/{}.csv'.format( filename ) )

In [147]:
del meta[ 'Unnamed: 0' ]

In [148]:
meta.shape

(1120, 11)

In [149]:
meta.dtypes

replicate      int64
time           int64
S            float64
E            float64
A            float64
I1           float64
I2           float64
I3           float64
R            float64
D            float64
obs          float64
dtype: object

In [150]:
meta.head( 20 )

,replicate,time,S,E,A,I1,I2,I3,R,D,obs
0,1,0,1.028736e+06,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,1,1.028735e+06,1.195159,0.114864,1.741960,-0.012052,-0.000307,0.190094,-0.000005,NaN
2,1,2,1.028733e+06,2.292329,0.417999,1.523373,-0.013150,-0.000902,0.377022,-0.000036,NaN
3,1,3,1.028731e+06,3.548624,0.892057,1.339287,0.002206,-0.001130,0.583224,-0.000090,NaN
4,1,4,1.028729e+06,5.180990,1.563858,1.186585,0.039308,-0.000161,0.831834,-0.000129,NaN
5,1,5,1.028725e+06,7.422905,2.496911,1.063688,0.104763,0.003021,1.150851,-0.000069,NaN
6,1,6,1.028720e+06,10.568365,3.793790,0.970565,0.207785,0.009691,1.577200,0.000231,0.0
7,1,7,1.028714e+06,15.015921,5.606027,0.908947,0.361628,0.021505,2.161543,0.000986,0.0
8,1,8,1.028704e+06,21.321962,8.151440,0.882772,0.585413,0.040692,2.974636,0.002504,0.0
9,1,9,1.028690e+06,30.271698,11.740374,0.898879,0.906616,0.070314,4.116103,0.005228,0.0


In [151]:
projections = meta[ [ 'replicate', 'time', 'obs' ] ].fillna( 0 ).merge( time, how = 'left', on = 'time' )
projections.rename( columns = { 'obs': 'fallecidos' }, inplace = True )
projections = projections.loc[ projections[ 'date' ].notnull() ]
projections.to_csv( './meta-viz/data/{}-meta.csv'.format( city ), index = False )

In [127]:
meta.groupby( 'time' ).agg( { 'D': [ 'min', 'mean', 'median', 'max', percentile( 25 ), percentile( 75 ) ], 'obs': [ 'min', 'mean', 'median', 'max', percentile( 25 ), percentile( 75 ) ] } ).tail( 10 )

D                                                    \
             min        mean      median         max percentile_25   
time                                                                 
102   515.618142  515.618142  515.618142  515.618142    515.618142   
103   534.180310  534.180310  534.180310  534.180310    534.180310   
104   553.971866  553.971866  553.971866  553.971866    553.971866   
105   575.288599  575.288599  575.288599  575.288599    575.288599   
106   598.486589  598.486589  598.486589  598.486589    598.486589   
107   623.990334  623.990334  623.990334  623.990334    623.990334   
108   652.302056  652.302056  652.302056  652.302056    652.302056   
109   684.012182  684.012182  684.012182  684.012182    684.012182   
110   719.810860  719.810860  719.810860  719.810860    719.810860   
111   760.500315  760.500315  760.500315  760.500315    760.500315   

                      obs                                                   
     percentile_75    min   mean median    max percentile_25 percentile_75  
time                                                                        
102     515.618142  492.0  522.3  519.0  584.0        504.25        530.50  
103     534.180310  513.0  544.0  547.5  567.0        541.25        552.75  
104     553.971866  524.0  551.0  546.0  592.0        538.50        554.50  
105     575.288599  558.0  594.3  593.0  640.0        577.50        602.75  
106     598.486589  557.0  596.0  596.0  652.0        576.25        610.50  
107     623.990334  602.0  631.9  632.0  668.0        611.25        648.75  
108     652.302056  589.0  646.2  637.0  699.0        633.00        664.00  
109     684.012182  652.0  702.3  702.5  767.0        678.00        722.75  
110     719.810860  697.0  729.1  723.0  788.0        703.50        740.00  
111     760.500315  716.0  765.3  764.0  805.0        755.00        778.75

In [ ]:
to_export = meta.loc[ meta[ 'replicate' ] == 1 ][ [ 'time', 'I2', 'I3', 'D' ] ].round( 0 )

In [ ]:
to_export[ 'I2' ] = to_export[ 'I2' ].astype( int )
to_export[ 'I3' ] = to_export[ 'I3' ].astype( int )
to_export[ 'D' ] = to_export[ 'D' ].astype( int )

In [ ]:
to_export.tail()

In [ ]:
to_export = to_export.iloc[ 1: ]

In [ ]:
to_export.tail()

In [12]:
real = pd.read_csv( './data/meta/old2/BOG17jun.csv' )

In [13]:
real.shape

(96, 3)

In [14]:
real[ 'date' ] = pd.to_datetime( real[ 'date' ], format = '%d/%m/%Y' )

In [15]:
real.dtypes

date    datetime64[ns]
time             int64
obs              int64
dtype: object

In [17]:
real.head()

,date,time,obs
0,2020-03-14,1,0
1,2020-03-15,2,0
2,2020-03-16,3,0
3,2020-03-17,4,0
4,2020-03-18,5,0


In [ ]:
to_export = real.merge( to_export, how = 'outer', on = 'time' )

In [ ]:
to_export.rename( columns = { 'date': 'Fecha', 'obs': 'Fallecidos_reales', 'D': 'Fallecidos', 'I2': 'Graves', 'I3': 'Críticos' }, inplace = True )

In [ ]:
def impute_date( x ):
    if pd.isnull( x[ 'Fecha' ] ):
        return max( to_export[ 'Fecha' ] ) + pd.DateOffset( x[ 'time' ] - 96 )
    else:
        return x[ 'Fecha' ]
    
to_export[ 'Fecha' ] = to_export.apply( lambda x: impute_date( x ), axis = 1 )

In [ ]:
to_export.tail( 25 )

In [ ]:
to_export = pd.melt( to_export, id_vars=[ 'Fecha' ], value_vars = [ 'Fallecidos_reales', 'Graves', 'Críticos', 'Fallecidos' ] )

In [ ]:
#to_export.drop_duplicates( subset = [ 'value' ], inplace = True )

In [ ]:
to_export.head()

In [ ]:
to_export.to_csv( './agents-viz/data/meta.csv', index = False )